# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [6]:
driver = webdriver.Chrome()
driver.get('https://arlweb.msha.gov/drs/drshome.htm')
mineid = driver.find_element_by_name('MineId')
mineid.send_keys('3503598')
button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
button.click()

### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [9]:
Bdate = driver.find_element_by_name('BDate')
Bdate.send_keys('1/1/1995')
vio_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
vio_button.click()
getrep_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
getrep_button.click()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [15]:
rows = driver.find_elements_by_css_selector('tr.drsviols')
datapoints = []
for row in rows:
    cells = row.find_elements_by_css_selector('td.drsviols')
    data = {}
    data['violator'] = cells[0].text
    data['contract_id'] = row.find_element_by_css_selector('font.drsviols').text
    data['citation_no'] = cells[1].text
    data['case_no'] = cells[2].text
    data['date_issues'] = cells[3].text
    data['final_order_date'] = cells[4].text
    data['section_of_act'] = cells[5].text
    data['date_terminated'] = cells[6].text
    data['citation'] = cells[7].text
    data['s_and_s'] = cells[8].text
    data['standard'] = cells[9].text
    data['standard_url'] = cells[9].find_element_by_tag_name('a').get_attribute('href')
    data['proposed_penalty'] = cells[10].text
    data['citation_status'] = cells[11].text
    data['current_penalty'] = cells[12].text
    data['amount_paid'] = cells[13].text
    datapoints.append(data)
datapoints

[{'violator': 'Newberg Rock & Dirt',
  'contract_id': '  ',
  'citation_no': '8992368  ',
  'case_no': '000440809',
  'date_issues': '5/2/2017',
  'final_order_date': '7/23/2017 ',
  'section_of_act': '104(a)',
  'date_terminated': '5/2/2017',
  'citation': 'C',
  's_and_s': 'N',
  'standard': '56.14107(a)',
  'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf',
  'proposed_penalty': '116.00',
  'citation_status': 'Closed',
  'current_penalty': '116.00 ',
  'amount_paid': '116.00 '},
 {'violator': 'Newberg Rock & Dirt',
  'contract_id': '  ',
  'citation_no': '8992214  ',
  'case_no': '000435717',
  'date_issues': '2/7/2017',
  'final_order_date': '5/21/2017 ',
  'section_of_act': '104(a)',
  'date_terminated': '2/8/2017',
  'citation': 'C',
  's_and_s': 'N',
  'standard': '56.14107(a)',
  'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf',
  'proposed_penalty': '116.0

### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [16]:
import pandas as pd
df = pd.DataFrame(datapoints)
df.head()

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.00,000440809,C,8992368,Closed,,116.00,5/2/2017,5/2/2017,7/23/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
1,116.00,000435717,C,8992214,Closed,,116.00,2/7/2017,2/8/2017,5/21/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
2,100.00,000398910,C,8872461,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
3,100.00,000398910,C,8872462,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
4,100.00,000383840,C,8790965,Closed,,100.00,4/15/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt


In [17]:
df.to_csv("3503598-violations.csv", index=False)

## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

In [ ]:
#prep
driver = webdriver.Chrome()
driver.get('https://arlweb.msha.gov/drs/drshome.htm')
mineid = driver.find_element_by_name('MineId')
mineid.send_keys('3503598')
button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
button.click()

Bdate = driver.find_element_by_name('BDate')
Bdate.send_keys('1/1/1995')
vio_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
vio_button.click()
getrep_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
getrep_button.click()

#sraping results
rows = driver.find_elements_by_css_selector('tr.drsviols')
datapoints = []
for row in rows:
    cells = row.find_elements_by_css_selector('td.drsviols')
    data = {}
    data['violator'] = cells[0].text
    data['contract_id'] = row.find_element_by_css_selector('font.drsviols').text
    data['citation_no'] = cells[1].text
    data['case_no'] = cells[2].text
    data['date_issues'] = cells[3].text
    data['final_order_date'] = cells[4].text
    data['section_of_act'] = cells[5].text
    data['date_terminated'] = cells[6].text
    data['citation'] = cells[7].text
    data['s_and_s'] = cells[8].text
    data['standard'] = cells[9].text
    data['standard_url'] = cells[9].find_element_by_tag_name('a').get_attribute('href')
    data['proposed_penalty'] = cells[10].text
    data['citation_status'] = cells[11].text
    data['current_penalty'] = cells[12].text
    data['amount_paid'] = cells[13].text
    datapoints.append(data)
datapoints

#turning results into df
import pandas as pd
df = pd.DataFrame(datapoints)
df.head()

#saving to csv
df.to_csv("3503598-violations.csv", index=False)

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [20]:
df_subset = pd.read_csv('mines-subset.csv')
df_subset

,id
0,4104757
1,801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

In [21]:
df_subset = pd.read_csv('mines-subset.csv',dtype = 'str')
df_subset

,id
0,4104757
1,0801306
2,3609931


### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

In [43]:
def get_info(rowinfo):
    #prep
    
    driver.get('https://arlweb.msha.gov/drs/drshome.htm')
    mineid = driver.find_element_by_name('MineId')
    mineid.send_keys(rowinfo['id'])
    button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    button.click()

    Bdate = driver.find_element_by_name('BDate')
    Bdate.send_keys('1/1/1995')
    vio_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
    vio_button.click()
    getrep_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
    getrep_button.click()

    #sraping results
    rows = driver.find_elements_by_css_selector('tr.drsviols')
    datapoints = []
    for row in rows:
        cells = row.find_elements_by_css_selector('td.drsviols')
        data = {}
        data['violator'] = cells[0].text
        data['contract_id'] = row.find_element_by_css_selector('font.drsviols').text
        data['citation_no'] = cells[1].text
        data['case_no'] = cells[2].text
        data['date_issues'] = cells[3].text
        data['final_order_date'] = cells[4].text
        data['section_of_act'] = cells[5].text
        data['date_terminated'] = cells[6].text
        data['citation'] = cells[7].text
        data['s_and_s'] = cells[8].text
        try:
            data['standard'] = cells[9].text
            data['standard_url'] = cells[9].find_element_by_tag_name('a').get_attribute('href')
        except:
            data['standard'] = 'no info'
            data['standard_url'] = 'no url'
        try:
            data['proposed_penalty'] = cells[10].text
            data['citation_status'] = cells[11].text
            data['current_penalty'] = cells[12].text
            data['amount_paid'] = cells[13].text
            datapoints.append(data)
        except:
            continue
        
    datapoints

    #turning results into df
    import pandas as pd
    df = pd.DataFrame(datapoints)
    #saving to csv
    df.to_csv(rowinfo['id']+'-violations.csv', index=False)
    

    

In [31]:
df_subset.apply(get_info,axis=1)

0    None
1    None
2    None
dtype: object

In [34]:
#double check if i get the info right...
pd.set_option('display.max_colwidth', -1)
df_test = pd.read_csv(df_subset['id'][2]+'-violations.csv')
df_test

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.0,442816,C,9317668,Closed,,116.0,5/11/2017,5/11/2017,8/19/2017,116.0,N,104(a),50.30(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec50-30.pdf,M.R. Dirt Inc.
1,114.0,421654,C,8928850,Closed,,114.0,8/8/2016,8/8/2016,11/10/2016,114.0,N,104(a),56.9301,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-vol1/pdf/CFR-2016-title30-vol1-sec56-9301.pdf,M.R. Dirt Inc.
2,100.0,380669,C,8807882,Closed,,100.0,4/2/2015,4/2/2015,6/7/2015,100.0,N,104(a),56.14132(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-vol1/pdf/CFR-2015-title30-vol1-sec56-14132.pdf,M.R. Dirt Inc.
3,100.0,282555,C,8650963,Closed,,100.0,1/23/2012,1/24/2012,4/12/2012,100.0,N,104(a),56.14100(b),http://www.gpo.gov/fdsys/pkg/CFR-2012-title30-vol1/pdf/CFR-2012-title30-vol1-sec56-14100.pdf,M.R. Dirt Inc.
4,100.0,274355,C,8650926,Closed,,100.0,10/19/2011,10/20/2011,1/11/2012,100.0,N,104(a),56.1000,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-1000.pdf,M.R. Dirt Inc.
5,100.0,262818,C,8645898,Closed,,100.0,6/9/2011,6/9/2011,9/15/2011,100.0,N,104(a),56.4101,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4101.pdf,M.R. Dirt Inc.
6,108.0,262818,C,8645897,Closed,,108.0,6/9/2011,6/9/2011,9/15/2011,108.0,Y,104(a),56.3200,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-3200.pdf,M.R. Dirt Inc.
7,100.0,262818,C,8645896,Closed,,100.0,6/9/2011,6/14/2011,9/15/2011,100.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-14107.pdf,M.R. Dirt Inc.


# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.

In [44]:
df_all = pd.read_csv('mines.csv',dtype = 'str')
df_all

,id
0,3503598
1,4801789
2,5001797
3,4608254
4,2103723
5,4104757
6,0801306
7,3901432
8,3609624
9,3609931


In [46]:
driver = webdriver.Chrome()
df_all.apply(get_info,axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object